<a href="https://colab.research.google.com/github/acamposmcr/Python/blob/master/Limpieza_de_Base_de_datos_para_tiempos_de_digitaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparación de los datos para el análisis de los Tiempos de Digitación 

In [0]:
# importando librerías

import os
import ipywidgets as widgets
import numpy as np
import pandas as pd

ruta_nb=os.getcwd()
ruta_data=ruta_nb+'/Data/'

In [0]:
radb=widgets.RadioButtons(options=['CSV','EXCEL'],
                          description='Selección:',disabled=False)
print('Seleccione el tipo de archivo y luego presione [Shift]+[Enter] en la siguiente celda:')
display(radb)

Seleccione el tipo de archivo y luego presione [Shift]+[Enter] en la siguiente celda:


RadioButtons(description='Selección:', options=('CSV', 'EXCEL'), value='CSV')

In [0]:
# cargando el archivo de datos a la variable df

if radb.value == 'CSV':
    archivo=ruta_data+'Tiempos de digitación 2018.csv'
    df=pd.read_csv(archivo, delimiter=';',parse_dates=[0],dayfirst=True)
elif radb.value == 'EXCEL':
    archivo=ruta_data+'Tiempos de digitación 2018.xlsx'
    df=pd.read_excel(archivo,sheet_name='Hoja1')
print('Archivo de datos cargado en variable <df>')

Archivo de datos cargado en variable <df>


In [0]:
# exploración inicial

print('\nColumnas:',df.columns.to_list())
print('\nFilas:',df.shape[0])
print(df['Evento'].value_counts().reindex(['Login','InicioDataEntry','FinDataEntry1','FinDataEntry2']).to_frame())
print('\n\tTotal: ',df['Evento'].count())


Columnas: ['Fecha', 'Hora', 'Evento', 'Num Muestra', 'Segundo Data', 'Usuario']

Filas: 378446
                 Evento
Login             87768
InicioDataEntry  152533
FinDataEntry1     67195
FinDataEntry2     70950

	Total:  378446


In [0]:
# separando InicioDataEntry en InicioDataEntry1 e InicioDataEntry2

df.loc[(df['Evento']=='InicioDataEntry')&(df['Segundo Data']=='DBDE'),'Evento']='InicioDataEntry2'
df.loc[(df['Evento']=='InicioDataEntry'),'Evento']='InicioDataEntry1'
orden=['Login','InicioDataEntry1','InicioDataEntry2','FinDataEntry1','FinDataEntry2']
print(df['Evento'].value_counts().reindex(orden).to_frame())
print('\n\tTotal: ',df['Evento'].count())

                  Evento
Login              87768
InicioDataEntry1   75534
InicioDataEntry2   76999
FinDataEntry1      67195
FinDataEntry2      70950

	Total:  378446


In [0]:
# creando columna nueva [Fecha_hora] y convirtiendo a tipo datetime

print('\nCreando nueva columna y convirtiendo a tipo datetime...')
df['Fecha_Hora'] = df['Fecha'].astype(str) + ' ' + df['Hora'].astype(str)
df['Fecha_Hora'] = df['Fecha_Hora'].apply(pd.to_datetime)
print('\nColumnas:',df.columns.to_list())
print('\nFilas:',df.shape[0])


Creando nueva columna y convirtiendo a tipo datetime...

Columnas: ['Fecha', 'Hora', 'Evento', 'Num Muestra', 'Segundo Data', 'Usuario', 'Fecha_Hora']

Filas: 378446


In [0]:
# verificando valores totales y duplicados en los valores de la columna [Evento]

orden=['Login','InicioDataEntry1','InicioDataEntry2','FinDataEntry1','FinDataEntry2']
tTotales=0
tSinDupl=0
print()
for evento in df['Evento'].value_counts().reindex(orden).index.tolist():
    totales=df[(df['Evento']==evento)]['Num Muestra'].value_counts().sum()
    tTotales+=totales
    sin_duplicados=df[(df['Evento']==evento)]['Num Muestra'].value_counts().count()
    tSinDupl+=sin_duplicados
    print('De [{}] hay {} filas, sin duplicados hay {} filas'.format(evento,totales,sin_duplicados))
print('\nEl total de filas en la columna[Eventos] es de {}, sin duplicados es de {}'
      .format(tTotales,tSinDupl))  


De [Login] hay 87768 filas, sin duplicados hay 75234 filas
De [InicioDataEntry1] hay 75534 filas, sin duplicados hay 75178 filas
De [InicioDataEntry2] hay 76999 filas, sin duplicados hay 75143 filas
De [FinDataEntry1] hay 67195 filas, sin duplicados hay 64063 filas
De [FinDataEntry2] hay 70950 filas, sin duplicados hay 69026 filas

El total de filas en la columna[Eventos] es de 378446, sin duplicados es de 358644


In [0]:
# borrando duplicados en la columna [Evento]

for evento in orden:
    tmp=df[(df['Evento']==evento)]['Num Muestra'].value_counts().to_frame()
    tmp.columns=['cuenta']
    listado=tmp[(tmp['cuenta']>1)].index.to_list()
    listado2=tmp[(tmp['cuenta']==1)].index.to_list()
    cont=0
    print('Para <{}> se tienen {} con duplicados, {} sin duplicados, para un total de {}'.
         format(evento,len(listado),len(listado2),(len(listado)+len(listado2))))
    
    prog=widgets.IntProgress(
    value=0,
    min=1,
    max=len(listado),
    step=1,
    description='Borrando:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal')
    display(prog)
    
    for num in listado:
        cont+=1
        max_id = df[(df['Evento']==evento)&(df['Num Muestra']==num)].index.max()
        df.drop(df[(df['Evento']==evento)&(df['Num Muestra']==num)&(df.index != max_id)].index,inplace=True)
        prog.value=cont
    

Para <Login> se tienen 11868 con duplicados, 63366 sin duplicados, para un total de 75234


IntProgress(value=1, description='Borrando:', max=11868, min=1)

Para <InicioDataEntry1> se tienen 308 con duplicados, 74870 sin duplicados, para un total de 75178


IntProgress(value=1, description='Borrando:', max=308, min=1)

Para <InicioDataEntry2> se tienen 1104 con duplicados, 74039 sin duplicados, para un total de 75143


IntProgress(value=1, description='Borrando:', max=1104, min=1)

Para <FinDataEntry1> se tienen 2358 con duplicados, 61705 sin duplicados, para un total de 64063


IntProgress(value=1, description='Borrando:', max=2358, min=1)

Para <FinDataEntry2> se tienen 1814 con duplicados, 67212 sin duplicados, para un total de 69026


IntProgress(value=1, description='Borrando:', max=1814, min=1)

In [0]:
orden=['Login','InicioDataEntry1','InicioDataEntry2','FinDataEntry1','FinDataEntry2']
tTotales=0
tSinDupl=0
print()
for evento in df['Evento'].value_counts().reindex(orden).index.tolist():
    totales=df[(df['Evento']==evento)]['Num Muestra'].value_counts().sum()
    tTotales+=totales
    sin_duplicados=df[(df['Evento']==evento)]['Num Muestra'].value_counts().count()
    tSinDupl+=sin_duplicados
    print('De [{}] hay {} filas, sin duplicados hay {} filas'.format(evento,totales,sin_duplicados))
print('\nEl total de filas en la columna[Eventos] es de {}, sin duplicados es de {}'.format(tTotales,tSinDupl))  


De [Login] hay 75234 filas, sin duplicados hay 75234 filas
De [InicioDataEntry1] hay 75178 filas, sin duplicados hay 75178 filas
De [InicioDataEntry2] hay 75143 filas, sin duplicados hay 75143 filas
De [FinDataEntry1] hay 64063 filas, sin duplicados hay 64063 filas
De [FinDataEntry2] hay 69026 filas, sin duplicados hay 69026 filas

El total de filas en la columna[Eventos] es de 358644, sin duplicados es de 358644


In [0]:
df.to_csv(ruta_data+'Tiempo_Digitación_Modif.csv',index=False)